# Highly Scalable Efficient Recommendation System

## Dowload dataset from open data source
There are many data sets in two selected data sources, but we only need some of them to build our recommendation system. So we select a few of them in consideration of the demand for data in our system.

#### IMDb database
* title.basics.tsv
* title.crew.tsv
* title.ratings.tsv

#### MovieLens Data Sets
* tags.csv
* ratings.csv
* genome-scores.csv
* genome-tags.csv

In [8]:
import pandas as pd
import numpy as np

basics = pd.read_csv("Original_Data_Sets/IMDb/title.basics.tsv", sep='\t', header=0)
crew = pd.read_csv("Original_Data_Sets/IMDb/title.crew.tsv", sep='\t', header=0)
ratings = pd.read_csv("Original_Data_Sets/IMDb/title.ratings.tsv", sep='\t', header=0)

basics.head(5)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


In [9]:
links = pd.read_csv("Original_Data_Sets/MovieLens/links.csv", sep=",", header=0)
tags = pd.read_csv("Original_Data_Sets/MovieLens/tags.csv", sep=",", header=0)
user_ratings = pd.read_csv("Original_Data_Sets/MovieLens/ratings.csv", sep=",", header=0)
genome_scores = pd.read_csv("Original_Data_Sets/MovieLens/genome-scores.csv", sep=",", header=0)
genome_tag = pd.read_csv("Original_Data_Sets/MovieLens/genome-tags.csv", sep=",", header=0)

links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
imdbId     27278 non-null int64
tmdbId     27026 non-null float64
dtypes: float64(1), int64(2)
memory usage: 639.4 KB


In [4]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4941637 entries, 0 to 4941636
Data columns (total 9 columns):
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult           int64
startYear         object
endYear           object
runtimeMinutes    object
genres            object
dtypes: int64(1), object(8)
memory usage: 339.3+ MB


There are 27278 movies in MovieLens datset and 4941637 records in IMDb dataset.

## Data Integration
In IMDb database, the information of one movie is divided into three part, such as basic information of movie, information of crew and audience evaluation. So we combine three data sets into one data set according to the titleId of movies.  

* Form a large data set of 11420 movies with 12 attributes

In [101]:
extra_info = pd.merge(left=crew, right=ratings, how='outer', on='tconst')
movie = pd.merge(left=basics, right=extra_info, how='outer', on='tconst')
movie.to_csv("Data_Set/movie.csv")
movie.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",nm0005690,\N,5.8,1364.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",nm0721526,\N,6.5,160.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",nm0721526,\N,6.6,948.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",nm0721526,\N,6.4,96.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short,nm0005690,\N,6.2,1643.0


IMDb and MovieLens use different identity number to represent movies, we can two datasets using links.csv.

* Transform the attribute "tconst" in IMDb to the form of "imdbId" in MovieLens

In [102]:
movie['tconst'] = movie.apply(lambda x: int(x['tconst'].replace("tt", "")), axis=1)
movie.rename(columns=lambda x:x.replace('tconst', 'imdbId'), inplace=True)
movie.head(5)

,imdbId,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",nm0005690,\N,5.8,1364.0
1,2,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",nm0721526,\N,6.5,160.0
2,3,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",nm0721526,\N,6.6,948.0
3,4,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",nm0721526,\N,6.4,96.0
4,5,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short,nm0005690,\N,6.2,1643.0


## Data Reduction
### Feature Selection

In [104]:
movie = movie.replace('\\N', np.nan)
pd.isnull(movie).sum()

imdbId                  0
titleType               0
primaryTitle            4
originalTitle         180
isAdult                 0
startYear          276921
endYear           4902775
runtimeMinutes    3404085
genres             390533
directors         2120881
writers           2505777
averageRating     4117413
numVotes          4117413
dtype: int64

So we drop attributes as below:

* primaryTitle
* originalTitle
* endYear

In [105]:
movie = movie.drop(columns=['primaryTitle', 'originalTitle', 'endYear'])
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,short,0,1894,1,"Documentary,Short",nm0005690,NaN,5.8,1364.0
1,2,short,0,1892,5,"Animation,Short",nm0721526,NaN,6.5,160.0
2,3,short,0,1892,4,"Animation,Comedy,Romance",nm0721526,NaN,6.6,948.0
3,4,short,0,1892,NaN,"Animation,Short",nm0721526,NaN,6.4,96.0
4,5,short,0,1893,1,Short,nm0005690,NaN,6.2,1643.0


### Instance Selection

Here, we drop all videos and video games which are unrelated.

In [114]:
movie = movie.drop(np.where((movie.titleType == 'video') | (movie.titleType == 'videoGame'))[0])
movie.to_csv("Data_Set/movie_l1.csv", index=False)
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4722816 entries, 0 to 4941636
Data columns (total 10 columns):
imdbId            int64
titleType         object
isAdult           int64
startYear         float64
runtimeMinutes    object
genres            object
directors         object
writers           object
averageRating     float64
numVotes          float64
dtypes: float64(3), int64(2), object(5)
memory usage: 396.4+ MB


## Data Cleaning
Here, we handle missing data in IMDb dataset.

In [108]:
movie = pd.read_csv("Data_Set/movie_l1.csv", sep=",", header=0)

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
movie.groupby('titleType').mean()

,imdbId,isAdult,startYear,averageRating,numVotes
titleType,,,,,
movie,2.207283e+06,0.017511,1986.252302,6.257489,3137.623713
short,3.175621e+06,0.001691,1989.339339,6.875474,61.364852
tvEpisode,3.768220e+06,0.022563,2003.045081,7.449162,124.768424
tvMiniSeries,3.917497e+06,0.003608,2004.946546,7.267308,567.484533
tvMovie,2.400234e+06,0.000562,1995.310781,6.712848,176.661691
tvSeries,2.973238e+06,0.003968,2002.112311,6.836243,1078.392503
tvShort,2.532572e+06,0.000240,1993.065176,6.924822,179.432171
tvSpecial,2.056499e+06,0.000560,2001.810498,6.724496,155.683419


We use the mean value in same "titleType" to impute the missing values in "runtimeMinutes".

In [116]:
movie.loc[4805574, 'isAdult'] = 0
movie.loc[4805574, 'startYear'] = 2015
movie.loc[4805574, 'runtimeMinutes'] = np.nan
movie.loc[4805574, 'genres'] = 'Animation,Sci-Fi,Short'
movie.loc[4730468, 'isAdult'] = 0
movie.loc[4730468, 'startYear'] = 2018
movie.loc[4730468, 'runtimeMinutes'] = np.nan
movie.loc[4730468, 'genres'] = 'Animation,Family'
movie.loc[4525055, 'isAdult'] = 0
movie.loc[4525055, 'startYear'] = 2016
movie.loc[4525055, 'runtimeMinutes'] = np.nan
movie.loc[4525055, 'genres'] = 'Action,Adventure,Animation'
movie.loc[2301207, 'isAdult'] = 0
movie.loc[2301207, 'startYear'] = 2011
movie.loc[2301207, 'runtimeMinutes'] = np.nan
movie.loc[2301207, 'genres'] = 'Comedy'
movie.loc[811687, 'isAdult'] = 0
movie.loc[811687, 'startYear'] = 1993
movie.loc[811687, 'runtimeMinutes'] = np.nan
movie.loc[811687, 'genres'] = 'Action,Animation,Comedy'

movie['runtimeMinutes'] = movie['runtimeMinutes'].astype('float')
means = movie.groupby('titleType').mean()

movie['runtimeMinutes'] = movie.apply(
    lambda x: means.loc[x['titleType'], 'runtimeMinutes'] if np.isnan(x['runtimeMinutes']) else x['runtimeMinutes'],
    axis=1)

We use the mean value to impute the missing values in "averageRating".

In [117]:
movie['averageRating'] = movie['averageRating'].fillna(movie['averageRating'].mean())

We use the mean value in same "titleType" to impute the missing values in "numVotes".

In [118]:
movie['numVotes'] = movie.apply(
    lambda x: means.loc[x['titleType'], 'numVotes'] if np.isnan(x['numVotes']) else x['numVotes'],
    axis=1)

For other categorical data, we don't need to handle the missing data.

In [119]:
movie.head(10)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,short,0,1894.0,1.000000,"Documentary,Short",nm0005690,NaN,5.8,1364.0
1,2,short,0,1892.0,5.000000,"Animation,Short",nm0721526,NaN,6.5,160.0
2,3,short,0,1892.0,4.000000,"Animation,Comedy,Romance",nm0721526,NaN,6.6,948.0
3,4,short,0,1892.0,13.213336,"Animation,Short",nm0721526,NaN,6.4,96.0
4,5,short,0,1893.0,1.000000,Short,nm0005690,NaN,6.2,1643.0
5,6,short,0,1894.0,1.000000,Short,nm0005690,NaN,5.6,83.0
6,7,short,0,1894.0,1.000000,"Short,Sport","nm0374658,nm0005690",NaN,5.5,553.0
7,8,short,0,1894.0,1.000000,"Documentary,Short",nm0005690,NaN,5.6,1465.0
8,9,movie,0,1894.0,45.000000,Romance,nm0085156,nm0085156,5.5,65.0
9,10,short,0,1895.0,1.000000,"Documentary,Short",nm0525910,NaN,6.9,4910.0


## Data Transformation

For titleType, we use numbers 0-4 to update values.
* 0 if the titleType is movie or tvMovie
* 1 if the titleType is short or tvShort
* 2 if the titleType is tvEpisode or tvSeries
* 3 if the titleType is tvMiniSeries or tvSpecial

In [120]:
movie['titleType'] = np.where((movie.titleType == 'movie') | (movie.titleType == 'tvMovie'), 0, movie.titleType)
movie['titleType'] = np.where((movie.titleType == 'short') | (movie.titleType == 'tvShort'), 1, movie.titleType)
movie['titleType'] = np.where((movie.titleType == 'tvEpisode') | (movie.titleType == 'tvSeries'), 2, movie.titleType)
movie['titleType'] = np.where((movie.titleType == 'tvMiniSeries') | (movie.titleType == 'tvSpecial'), 3, movie.titleType)
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,1,0,1894.0,1.000000,"Documentary,Short",nm0005690,NaN,5.8,1364.0
1,2,1,0,1892.0,5.000000,"Animation,Short",nm0721526,NaN,6.5,160.0
2,3,1,0,1892.0,4.000000,"Animation,Comedy,Romance",nm0721526,NaN,6.6,948.0
3,4,1,0,1892.0,13.213336,"Animation,Short",nm0721526,NaN,6.4,96.0
4,5,1,0,1893.0,1.000000,Short,nm0005690,NaN,6.2,1643.0


For startYear, we divde them to 7 ages.
* 0 if the startYear is earlier than 1930
* 1 if the startYear is later than 1930 but earlier than 1950
* 2 if the startYear is later than 1950 but earlier than 1970
* 3 if the startYear is later than 1970 but earlier than 1990
* 4 if the startYear is later than 1990 but earlier than 2000
* 5 if the startYear is later than 2000 but earlier than 2010
* 6 if the startYear is later than 2010

In [123]:
movie['startYear'] = np.where(movie.startYear <= 1930, 0, movie.startYear)
movie['startYear'] = np.where((movie.startYear > 1930) & (movie.startYear <= 1950), 1, movie.startYear)
movie['startYear'] = np.where((movie.startYear > 1950) & (movie.startYear <= 1970), 2, movie.startYear)
movie['startYear'] = np.where((movie.startYear > 1970) & (movie.startYear <= 1990), 3, movie.startYear)
movie['startYear'] = np.where((movie.startYear > 1990) & (movie.startYear <= 2000), 4, movie.startYear)
movie['startYear'] = np.where((movie.startYear > 2000) & (movie.startYear <= 2010), 5, movie.startYear)
movie['startYear'] = np.where(movie.startYear > 2010, 6, movie.startYear)
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,1,0,0.0,1.000000,"Documentary,Short",nm0005690,NaN,5.8,1364.0
1,2,1,0,0.0,5.000000,"Animation,Short",nm0721526,NaN,6.5,160.0
2,3,1,0,0.0,4.000000,"Animation,Comedy,Romance",nm0721526,NaN,6.6,948.0
3,4,1,0,0.0,13.213336,"Animation,Short",nm0721526,NaN,6.4,96.0
4,5,1,0,0.0,1.000000,Short,nm0005690,NaN,6.2,1643.0


For runtimeMinutes, we divide them to 5 types.
* 0 if the runtimeMinutes is less than 10
* 1 if the runtimeMinutes is larger than 10 but less than 30
* 2 if the runtimeMinutes is larger than 30 but less than 60
* 3 if the runtimeMinutes is larger than 60 but less than 120
* 4 if the runtimeMinutes is larger than 120

In [125]:
movie['runtimeMinutes'] = np.where(movie.runtimeMinutes <= 10, 0, movie.runtimeMinutes)
movie['runtimeMinutes'] = np.where((movie.runtimeMinutes > 10) & (movie.runtimeMinutes <= 30), 1, movie.runtimeMinutes)
movie['runtimeMinutes'] = np.where((movie.runtimeMinutes > 30) & (movie.runtimeMinutes <= 60), 2, movie.runtimeMinutes)
movie['runtimeMinutes'] = np.where((movie.runtimeMinutes > 60) & (movie.runtimeMinutes <= 120), 3, movie.runtimeMinutes)
movie['runtimeMinutes'] = np.where(movie.runtimeMinutes > 120, 4, movie.runtimeMinutes)
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,1,0,0.0,0.0,"Documentary,Short",nm0005690,NaN,5.8,1364.0
1,2,1,0,0.0,0.0,"Animation,Short",nm0721526,NaN,6.5,160.0
2,3,1,0,0.0,0.0,"Animation,Comedy,Romance",nm0721526,NaN,6.6,948.0
3,4,1,0,0.0,1.0,"Animation,Short",nm0721526,NaN,6.4,96.0
4,5,1,0,0.0,0.0,Short,nm0005690,NaN,6.2,1643.0


For averageRating, we divide them to 5 types.
* 0 if the averageRating is less than 5.0
* 1 if the averageRating is larger than 5.0 but less than 6.0
* 2 if the averageRating is larger than 6.0 but less than 7.0
* 3 if the averageRating is larger than 7.0 but less than 8.0
* 4 if the averageRating is larger than 8.0

In [127]:
movie['averageRating'] = np.where(movie.averageRating <= 5.0, 0, movie.averageRating)
movie['averageRating'] = np.where((movie.averageRating > 5.0) & (movie.averageRating <= 6.0), 1, movie.averageRating)
movie['averageRating'] = np.where((movie.averageRating > 6.0) & (movie.averageRating <= 7.0), 2, movie.averageRating)
movie['averageRating'] = np.where((movie.averageRating > 7.0) & (movie.averageRating <= 8.0), 3, movie.averageRating)
movie['averageRating'] = np.where(movie.averageRating > 8.0, 4, movie.averageRating)
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,1,0,0.0,0.0,"Documentary,Short",nm0005690,NaN,1.0,1364.0
1,2,1,0,0.0,0.0,"Animation,Short",nm0721526,NaN,2.0,160.0
2,3,1,0,0.0,0.0,"Animation,Comedy,Romance",nm0721526,NaN,2.0,948.0
3,4,1,0,0.0,1.0,"Animation,Short",nm0721526,NaN,2.0,96.0
4,5,1,0,0.0,0.0,Short,nm0005690,NaN,2.0,1643.0


For numVotes, we divide them to 5 types.
* 0 if the numVotes is less than 100
* 1 if the numVotes is larger than 100 but less than 1000
* 2 if the numVotes is larger than 1000 but less than 10000
* 3 if the numVotes is larger than 10000 but less than 100000
* 4 if the numVotes is larger than 100000

In [137]:
movie['numVotes'] = np.where(movie.numVotes <= 100.0, 0, movie.numVotes)
movie['numVotes'] = np.where((movie.numVotes > 100.0) & (movie.numVotes <= 1000.0), 1, movie.numVotes)
movie['numVotes'] = np.where((movie.numVotes > 1000.0) & (movie.numVotes <= 10000.0), 2, movie.numVotes)
movie['numVotes'] = np.where((movie.numVotes > 10000.0) & (movie.numVotes <= 100000.0), 3, movie.numVotes)
movie['numVotes'] = np.where(movie.numVotes > 100000.0, 4, movie.numVotes)
movie.head(5)

,imdbId,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,1,1,0,0.0,0.0,"Documentary,Short",nm0005690,NaN,1.0,2.0
1,2,1,0,0.0,0.0,"Animation,Short",nm0721526,NaN,2.0,1.0
2,3,1,0,0.0,0.0,"Animation,Comedy,Romance",nm0721526,NaN,2.0,1.0
3,4,1,0,0.0,1.0,"Animation,Short",nm0721526,NaN,2.0,0.0
4,5,1,0,0.0,0.0,Short,nm0005690,NaN,2.0,2.0


In [138]:
movie.to_csv("Data_Set/movie_l2.csv", index=False)

## Build Movie-Tag Matrix

Movie-Tag Matrix is defined as the representation of movies using their relevance score with tags, where each row represents one movie, each column represents one tag. Thus, the value Mij is the relevance score between movie i and tag j. Further more, if there is no tag j on movie i, the value Mij was 0, which means the vector of a new movie is a zero vector.

In [13]:
movie_tag = pd.read_csv("Original_Data_Sets/MovieLens/genome-scores.csv", sep=",", header=0)
tags = pd.read_csv("Original_Data_Sets/MovieLens/genome-tags.csv", sep=",", header=0)
movie_num = links['movieId'].count()
tag_num = tags['tagId'].count()
movie_tag_matrix = np.zeros((movie_num, tag_num))
movie_tag_matrix.shape()

In [40]:
for a in range(movie_num):
    tag_relevance = movie_tag[movie_tag['movieId'] == links.loc[a, 'movieId']]
    if tag_relevance['tagId'].count() > 0:
        movie_tag_matrix[a][:] = list(tag_relevance['relevance'])
np.savetxt('movie_tag_matrix.csv', movie_tag_matrix, delimiter = ',')  

51
109
115
124
127
130
133
134
139
142
143
226
284
336
395
396
399
400
401
403
404
530
560
567
572
576
579
584
591
604
607
620
623
625
629
636
642
643
644
654
655
657
672
675
676
683
684
693
699
721
723
727
730
738
739
744
749
752
758
767
768
769
770
772
773
774
777
795
796
812
814
815
822
825
826
855
857
859
873
878
887
890
894
975
977
979
983
1001
1052
1065
1106
1109
1110
1115
1118
1122
1133
1139
1140
1141
1142
1145
1146
1155
1156
1157
1158
1159
1160
1163
1165
1166
1308
1309
1316
1319
1368
1386
1400
1421
1424
1428
1434
1443
1448
1462
1467
1490
1510
1522
1524
1528
1548
1553
1557
1558
1559
1561
1568
1577
1578
1579
1628
1630
1669
1692
1697
1705
1706
1708
1709
1710
1714
1716
1720
1724
1738
1740
1742
1764
1765
1768
1774
1776
1781
1782
1787
1789
1796
1815
1819
1830
1842
1843
1849
1851
1852
1905
1915
2030
2127
2213
2216
2217
2218
2220
2222
2223
2224
2225
2229
2230
2233
2235
2251
2254
2258
2270
2274
2277
2299
2308
2309
2319
2326
2343
2438
2477
2480
2484
2489
2508
2538
2547
2556
2564
2573
257

25977
25979
25980
25984
25986
25987
25989
25990
25991
25994
25998
26000
26001
26006
26008
26009
26010
26013
26016
26018
26019
26021
26022
26025
26026
26027
26028
26030
26033
26034
26035
26042
26043
26044
26046
26047
26048
26049
26050
26051
26056
26058
26059
26062
26063
26066
26067
26068
26070
26072
26073
26074
26076
26079
26080
26083
26086
26088
26090
26093
26095
26096
26098
26099
26100
26104
26106
26107
26108
26109
26110
26112
26113
26123
26125
26127
26128
26130
26136
26137
26140
26141
26142
26144
26146
26147
26153
26156
26157
26159
26160
26162
26164
26167
26169
26174
26175
26176
26177
26180
26183
26184
26185


KeyboardInterrupt: 